In [2]:
from modules.ingestion.chunking import llmsherpa, unstructed_io

name_file_pdf = 'finance_41pages.pdf'

text = llmsherpa.chunking_to_get_text(name_file_pdf)
table = unstructed_io.chunking_to_get_table_image(name_file_pdf, file_type='pdf')

c:\Users\TNI5HC\.conda\envs\multimodel_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.

In [6]:
list_text = [i['text'] for i in text]
list_table = [i['text'] for i in table]

list_text = list_text + list_table

In [7]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

model_url = "sentence-transformers/all-MiniLM-L6-v2"

model = AutoModel.from_pretrained(
    pretrained_model_name_or_path=model_url, device_map='cuda'
)

tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_url, device_map='duda'
)

In [8]:
from torch.utils.data import Dataset, DataLoader
from transformers import DataCollatorWithPadding

class ChunkDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        return {
            "input_ids": self.data["input_ids"][idx],
            "attention_mask": self.data["attention_mask"][idx],
        }

In [9]:
tokenized_data = tokenizer(list_text*10, max_length=256, truncation=True, padding=True, return_tensors='pt')


In [6]:
dataset = ChunkDataset(tokenized_data)

In [1]:
dataloader = DataLoader(dataset,
                        batch_size=32,
                        shuffle=False,
                        collate_fn=DataCollatorWithPadding(tokenizer))


NameError: name 'DataLoader' is not defined

In [14]:
for batch in dataloader:

    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')

    output = model(input_ids, attention_mask)


torch.Size([2, 14, 384])
torch.Size([2, 13, 384])
torch.Size([2, 32, 384])
torch.Size([2, 187, 384])
torch.Size([2, 203, 384])
torch.Size([2, 52, 384])
torch.Size([2, 67, 384])
torch.Size([2, 113, 384])
torch.Size([2, 200, 384])
torch.Size([2, 97, 384])
torch.Size([2, 120, 384])
torch.Size([2, 256, 384])
torch.Size([2, 256, 384])
torch.Size([2, 256, 384])
torch.Size([2, 256, 384])
torch.Size([2, 131, 384])
torch.Size([2, 256, 384])
torch.Size([2, 256, 384])
torch.Size([2, 256, 384])
torch.Size([2, 15, 384])
torch.Size([2, 238, 384])
torch.Size([2, 256, 384])
torch.Size([2, 256, 384])
torch.Size([2, 256, 384])
torch.Size([2, 256, 384])
torch.Size([2, 256, 384])
torch.Size([2, 35, 384])
torch.Size([2, 119, 384])
torch.Size([2, 14, 384])
torch.Size([2, 49, 384])
torch.Size([2, 59, 384])
torch.Size([2, 4, 384])
torch.Size([2, 45, 384])
torch.Size([2, 20, 384])
torch.Size([2, 19, 384])
torch.Size([2, 41, 384])
torch.Size([2, 11, 384])
torch.Size([2, 15, 384])
torch.Size([2, 6, 384])
torch.S

In [14]:
output = model(**tokenized_data)

In [15]:
len(output)

2